In [1]:
# !pip install --upgrade pip

In [2]:
# !pip install google-generativeai

In [3]:
# !pip install python-dotenv

# Gemini API 테스트

In [4]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
load_dotenv(".chatenv")

/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [5]:
GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [6]:
model = genai.GenerativeModel("gemini-1.5-flash")

In [7]:
response = model.generate_content("Write a story about a magic backpack.")
print(response.text)

Ten-year-old Finn was a master of finding trouble. He'd tripped over his own feet, spilled juice on his homework, and once even managed to get stuck in a tree while chasing a squirrel. His mother, weary of his antics, had finally snapped. "Finn, you need to be more careful!" she said, her voice laced with frustration. "I swear, you'd trip over a speck of dust!"

That's when she presented him with a peculiar gift: a battered, brown leather backpack. It was worn and patched, with a single, silver buckle in the shape of a crescent moon. "This is your responsibility now, Finn," she said. "Take care of it, and it will take care of you."

Finn wasn't thrilled. He'd much rather have a new video game, but he lugged the backpack to school anyway. In class, he felt a peculiar tingling sensation from the bag. Curiosity piqued, he reached inside and pulled out a shimmering, blue butterfly. It fluttered in his hand, then zipped out the window, leaving a trail of sparkling dust in its wake.

Over th

# Gemini 챗봇 만들기 1단계

In [8]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
import gradio as gr

# Load environment variables
load_dotenv(".chatenv")
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Define the function to generate content
def generate_response(prompt):
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    return response.text

# Create the Gradio app
with gr.Blocks() as app:
    gr.Markdown("# Chatbot with Google Generative AI")
    
    with gr.Tab("Chat"):
        chatbot_output = gr.Textbox(label="Chatbot Response", lines=10)
        user_input = gr.Textbox(label="Your Question")
        generate_btn = gr.Button("Generate Response")
        generate_btn.click(
            fn=generate_response, inputs=user_input, outputs=chatbot_output
        )

app.launch(inline=False, share=True)


/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")


Running on local URL:  http://127.0.0.1:7866
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://d7b5570fc69be3dd3f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# 네이버 뉴스 API 검색 코드 가져오기

In [11]:
import requests
import pandas as pd
from datetime import date
url = "https://openapi.naver.com/v1/search/news.json"
payload = {'query': '핀테크', 'display' : 100, 'start' : 1, 'sort': 'date'}
headers = {"X-Naver-Client-Id" : os.getenv('client_id'), "X-Naver-Client-Secret" : os.getenv('client_secret')}
r = requests.get(url, params=payload, headers=headers)
print(r.url)
if(r.status_code == 200):
    data = r.json()
    print(type(data))
else:
    print("Error Code:", r.status_code)
    today = date.today()
# 원하는 형식으로 변환
formatted_date = today.strftime("%d %b %Y")

print(formatted_date)  # 출력 예: 02 Aug 2024
def text_clean(x):
    x = x.replace("&quot;", "").replace("<b>", "").replace("</b>", "").replace("‘", "").replace("’", "")
    return x
result = {}
for item in data['items']:
    for key in item.keys():
        result.setdefault(key, []).append(text_clean(item[key]))
df = pd.DataFrame(result)
df

https://openapi.naver.com/v1/search/news.json?query=%ED%95%80%ED%85%8C%ED%81%AC&display=100&start=1&sort=date
<class 'dict'>


# 입력한 질문에서 키워드를 추출해 뉴스를 검색하고 선택한 뉴스를 요약해주는 챗봇

In [20]:
# !pip install nltk

In [17]:
# !pip install konlpy

In [27]:
# !pip install beautifulsoup4
# !pip install lxml

In [49]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
import gradio as gr
import requests
import pandas as pd
from datetime import date
from konlpy.tag import Okt
from nltk.util import ngrams
from bs4 import BeautifulSoup
import string

# 한국어 형태소 분석기 (Okt)
okt = Okt()

# Load environment variables
load_dotenv(".chatenv")
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
NAVER_CLIENT_ID = os.getenv('client_id')
NAVER_CLIENT_SECRET = os.getenv('client_secret')

genai.configure(api_key=GOOGLE_API_KEY)

# n-gram 키워드 추출 함수 (한국어용)
def extract_keywords_n_gram(text, n_range=(1, 2)):
    # 한국어 텍스트에서 명사 추출
    tokens = okt.nouns(text)  # 명사만 추출
    
    # 불필요한 구두점 필터링 (명사에서)
    filtered_tokens = [word for word in tokens if word not in string.punctuation]
    
    # 1-gram과 2-gram 추출
    keywords = []
    for n in range(n_range[0], n_range[1] + 1):
        n_grams = ngrams(filtered_tokens, n)
        for grams in n_grams:
            keywords.append(" ".join(grams))  # n-gram을 하나의 문자열로 변환
    
    # 추출된 키워드 반환
    return keywords[:5]  # 상위 5개 키워드만 반환

# Naver 뉴스 검색 함수 (앞서 설명한 대로)
def search_naver_news(query):
    url = "https://openapi.naver.com/v1/search/news.json"
    payload = {'query': query, 'display': 5, 'start': 1, 'sort': 'sim'}
    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
    }
    
    r = requests.get(url, params=payload, headers=headers)
    
    if r.status_code == 200:
        data = r.json()
        result = {}
        for item in data['items']:
            for key in item.keys():
                result.setdefault(key, []).append(item[key].replace("<b>", "").replace("</b>", "").replace("&quot;", ""))
        df = pd.DataFrame(result)
        return df
    else:
        return None

# Function to summarize news article from the link
def summarize_news(originallink):
    response = requests.get(originallink)
    if response.status_code == 200:
        # 여기서는 간단한 형태로 내용 수집 (실제로는 BeautifulSoup 등을 사용할 수 있습니다)
        content = response.text[:500]  # 간단하게 텍스트 일부를 보여줌
        # Gemini를 사용한 요약 (필요한 경우)
        model = genai.GenerativeModel("gemini-1.5-flash")
        summary = model.generate_content(content)
        return summary.text
    else:
        return "Failed to fetch the news."

# 수정된 generate_response_and_search_news 함수
def generate_response_and_search_news(prompt):
    # 1. Google Gemini를 이용한 응답 생성
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    response_text = response.text
    
    # 2. 질문에서 1-gram과 2-gram 키워드 추출 (한국어)
    keywords = extract_keywords_n_gram(prompt, n_range=(1, 2))
    search_query = " ".join(keywords)  # 추출된 키워드를 검색 쿼리로 변환
    
    # 3. 네이버 뉴스에서 추출된 키워드로 검색
    news_df = search_naver_news(search_query)
    
    if news_df is not None and not news_df.empty:
        # 뉴스 제목 리스트로 변환하여 Dropdown에 표시
        news_titles = news_df['title'].tolist()
        print(news_titles)
        return response_text, gr.Dropdown.update(choices=news_titles)  # 제목 리스트 반환 및 Dropdown 업데이트
    else:
        return response_text, gr.Dropdown.update(choices=[])

# 뉴스 본문에서 중요한 내용만 추출하는 함수
def extract_article_content(html):
    soup = BeautifulSoup(html, 'lxml')

    # 1. 기사 본문을 담고 있을 것으로 예상되는 태그 찾기
    # 일반적으로 기사 본문은 <div> 태그나 <p> 태그에 존재하며, 특정 클래스가 있을 수 있습니다.
    article_body = soup.find_all('p')  # <p> 태그 내부의 내용을 모두 가져오기

    # 2. 추출된 텍스트 결합
    article_text = ""
    for paragraph in article_body:
        article_text += paragraph.get_text() + " "

    return article_text.strip()

# 수정된 뉴스 요약 함수
def summarize_news(originallink):
    response = requests.get(originallink)
    
    if response.status_code == 200:
        # HTML 페이지 파싱
        article_content = extract_article_content(response.text)
        
        if article_content:
            # Gemini를 사용하여 본문 요약
            model = genai.GenerativeModel("gemini-1.5-flash")
            summary = model.generate_content(article_content)
            return summary.text
        else:
            return "No meaningful content found to summarize."
    else:
        return "Failed to fetch the news."

with gr.Blocks() as app:
    gr.Markdown("# Chatbot with Google Generative AI and Naver News Search")
    
    with gr.Tab("Chat and News Search"):
        with gr.Row():
            # Left side: Chatbot Response and User Input
            with gr.Column():
                chatbot_output = gr.Textbox(label="Chatbot Response", lines=10)
                user_input = gr.Textbox(label="Your Question")
                reset_btn = gr.Button("Reset")  # Reset button added here
                
            # Right side: News Articles and News Summary
            with gr.Column():
                news_list = gr.Dropdown(choices=[], label="News Articles", interactive=True)
                news_summary = gr.Textbox(label="News Summary", lines=10)
                view_news_btn = gr.Button("View News Summary")
        
        # Automatically trigger chatbot and news search on Enter key press
        user_input.submit(
            fn=generate_response_and_search_news, 
            inputs=user_input, 
            outputs=[chatbot_output, news_list]
        )
        
        # Button actions for viewing news summary
        view_news_btn.click(
            fn=get_news_summary, 
            inputs=news_list, 
            outputs=news_summary
        )
        
        # Reset button functionality to reset all fields including news_list
        reset_btn.click(
            fn=lambda: ("", "", [], ""),  # Reset all components
            inputs=[], 
            outputs=[user_input, chatbot_output, news_list, news_summary]
        )

app.launch(inline=False, share=True)


Running on local URL:  http://127.0.0.1:7889
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://f27cc5287ef57e5bab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


['[김주한 교수의 정보의료·디지털 사피엔스]챗GPT의 AI 앱스토어 전략', "[이원섭의 통통마컴] MS '빙'의 스미스 총괄사장이 한국에 온 까닭", '금융위기 재발 조짐… 경제전망 넘보는 구글AI']


In [40]:
app.close()

Closing server running on port: 7881


# 이전 대화 기억하는 챗봇

In [50]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
import gradio as gr
import requests
import pandas as pd
from datetime import date
from konlpy.tag import Okt
from nltk.util import ngrams
from bs4 import BeautifulSoup
import string

# 한국어 형태소 분석기 (Okt)
okt = Okt()

# Load environment variables
load_dotenv(".chatenv")
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
NAVER_CLIENT_ID = os.getenv('client_id')
NAVER_CLIENT_SECRET = os.getenv('client_secret')

genai.configure(api_key=GOOGLE_API_KEY)

# n-gram 키워드 추출 함수 (한국어용)
def extract_keywords_n_gram(text, n_range=(1, 2)):
    tokens = okt.nouns(text)  # 명사만 추출
    filtered_tokens = [word for word in tokens if word not in string.punctuation]
    
    keywords = []
    for n in range(n_range[0], n_range[1] + 1):
        n_grams = ngrams(filtered_tokens, n)
        for grams in n_grams:
            keywords.append(" ".join(grams))  # n-gram을 하나의 문자열로 변환
    
    return keywords[:5]  # 상위 5개 키워드만 반환

# Naver 뉴스 검색 함수
def search_naver_news(query):
    url = "https://openapi.naver.com/v1/search/news.json"
    payload = {'query': query, 'display': 5, 'start': 1, 'sort': 'sim'}
    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
    }
    
    r = requests.get(url, params=payload, headers=headers)
    
    if r.status_code == 200:
        data = r.json()
        result = {}
        for item in data['items']:
            for key in item.keys():
                result.setdefault(key, []).append(item[key].replace("<b>", "").replace("</b>", "").replace("&quot;", ""))
        df = pd.DataFrame(result)
        return df
    else:
        return None

# 뉴스 본문에서 중요한 내용만 추출하는 함수
def extract_article_content(html):
    soup = BeautifulSoup(html, 'lxml')
    article_body = soup.find_all('p')
    article_text = ""
    for paragraph in article_body:
        article_text += paragraph.get_text() + " "
    return article_text.strip()

# 뉴스 본문 요약 함수
def summarize_news(originallink):
    response = requests.get(originallink)
    
    if response.status_code == 200:
        article_content = extract_article_content(response.text)
        if article_content:
            model = genai.GenerativeModel("gemini-1.5-flash")
            summary = model.generate_content(article_content)
            return summary.text
        else:
            return "No meaningful content found to summarize."
    else:
        return "Failed to fetch the news."

# 대화 기록 리스트
conversation_history = []

# 대화 생성 및 뉴스 검색 함수
def generate_response_and_search_news(prompt):
    global conversation_history  # 전역 변수로 사용
    
    # 1. Google Gemini를 이용한 응답 생성
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    response_text = response.text
    
    # 대화 기록에 사용자 입력 및 Gemini 응답 추가
    conversation_history.append(f"User: {prompt}")
    conversation_history.append(f"Gemini: {response_text}")
    
    # 2. 질문에서 1-gram과 2-gram 키워드 추출 (한국어)
    keywords = extract_keywords_n_gram(prompt, n_range=(1, 2))
    search_query = " ".join(keywords)
    
    # 3. 네이버 뉴스에서 키워드로 검색
    news_df = search_naver_news(search_query)
    
    if news_df is not None and not news_df.empty:
        news_titles = news_df['title'].tolist()
        return "\n".join(conversation_history), gr.Dropdown.update(choices=news_titles), ""  # 입력 필드를 빈 문자열로 업데이트
    else:
        return "\n".join(conversation_history), gr.Dropdown.update(choices=[]), ""  # 입력 필드를 빈 문자열로 업데이트

# 전체 대화 기록 리셋 함수
def reset_conversation():
    global conversation_history
    conversation_history = []  # 대화 기록 초기화
    return "", "", [], ""  # 모든 필드 초기화

# 뉴스 요약 함수
def get_news_summary(selected_news):
    # selected_news는 뉴스 제목이므로, 데이터프레임에서 해당 제목의 링크를 찾아야 합니다.
    news_df = search_naver_news(selected_news)
    
    if news_df is not None and not news_df.empty:
        # selected_news와 제목이 일치하는 행에서 링크 추출
        news_row = news_df[news_df['title'] == selected_news]
        if not news_row.empty:
            # 'originallink' 컬럼에서 URL 추출
            originallink = news_row.iloc[0]['originallink']  # 첫 번째 행에서 링크 추출
            return summarize_news(originallink)
    
    return "No news found."

# Gradio UI
with gr.Blocks() as app:
    gr.Markdown("# Chatbot with Google Generative AI and Naver News Search")
    
    with gr.Tab("Chat and News Search"):
        with gr.Row():
            with gr.Column():
                chatbot_output = gr.Textbox(label="Chatbot Response", lines=10)
                user_input = gr.Textbox(label="Your Question")
                reset_btn = gr.Button("Reset")
                
            with gr.Column():
                news_list = gr.Dropdown(choices=[], label="News Articles", interactive=True)
                news_summary = gr.Textbox(label="News Summary", lines=10)
                view_news_btn = gr.Button("View News Summary")
        
        # 사용자 입력 시 대화 생성 및 뉴스 검색 후 입력 필드 초기화
        user_input.submit(
            fn=generate_response_and_search_news, 
            inputs=user_input, 
            outputs=[chatbot_output, news_list, user_input]  # user_input을 빈 문자열로 업데이트
        )
        
        # 뉴스 선택 시 요약 생성
        view_news_btn.click(
            fn=get_news_summary, 
            inputs=news_list, 
            outputs=news_summary
        )
        
        # Reset 버튼 클릭 시 대화 기록 및 필드 초기화
        reset_btn.click(
            fn=reset_conversation,
            inputs=[], 
            outputs=[user_input, chatbot_output, news_list, news_summary]
        )

app.launch(inline=False, share=True)


Running on local URL:  http://127.0.0.1:7890
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://03f1553cd15afb3310.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
